# Lasso Modeling!
### Yee-Haw!

As a control, I'm just gonna linearly regress right quick.

In [ ]:
import pandas as pd
import numpy as np

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('../data/processed/2 - Games DF - PreProcessed Features', 'rb') as file :
    X = pickle.load(file)

with open('../data/processed/2 - Games DF - PreProcessed Targets', 'rb') as file :
    y_universe = pickle.load(file)

%store -r relevant_langs

We're All Agnostics Now!
---

I'll choose the "agnostic" case for our initial analysis, and calculate basic vanilla lasso scores for all languages under that condition.

In [ ]:
# First, let me get a persistent set of indexes for my train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y_universe, test_size=0.2)

training_indexes = y_train.index
testing_indexes = y_test.index

In [ ]:
naive_scores = {}

for lang in relevant_langs :

    # Create the target variable for this lang
    y = {}
    for index, row in y_universe.iterrows() :
        y[index] = row['comment_diff_agnostic'][lang]
    y = pd.Series(y)

    # Generate our target splits based on the persistent train/test index split
    y_train = y[training_indexes]
    y_test = y[testing_indexes]

    

    # Run the model
    model = Lasso()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Score it and log it
    score = r2_score(y_test, y_pred)
    naive_scores[lang] = score
    print(f"{lang}: {round(score, 5)}")

The results are weak, but the vast majority did better than chance. That might mean our idea is not complete bunk.

Let's see if hyperparameter tuning helps.

In [ ]:
# First, define the function we'll use to tune our alpha
def Lasso_eval(alpha) :
    global X
    global y
    params = {"alpha":alpha}
    model = Lasso(max_iter=100000, **params)
    score = cross_val_score(model, X, y, cv=5).mean()
    return score


In [ ]:
# Now, do run the model again, optimizing on each lang

for lang in relevant_langs :

    # Create the target variable for this lang
    y = {}
    for index, row in y_universe.iterrows() :
        y[index] = row['comment_diff_agnostic'][lang]
    y = pd.Series(y)

    # Run some optimization
    param_bounds = {"alpha":(0, 100)}
    optimal = BayesianOptimization(Lasso_eval, param_bounds, verbose=False, allow_duplicate_points=True)
    optimal.maximize(60, 60)
    bayes_alpha = optimal.max['params']['alpha']

    # Generate our target splits based on the persistent train/test index split
    y_train = y[training_indexes]
    y_test = y[testing_indexes]

    # Run the model
    model = Lasso(max_iter=100000, alpha=bayes_alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Score it and log it
    score = r2_score(y_test, y_pred)
    rounded_score = round(score, 5)
    improvement = round(naive_scores[lang]-score, 6)
    rounded_alpha = round(optimal.max['params']['alpha'], 2)
    print(f"{lang}: {rounded_score} (improvement: {improvement}, alpha: {rounded_alpha})")

Nope.

I can only assume this is because the variability in the data makes the cv split scores so random that the tuned parameters end up essentially random as well.

So that's one strike AGAINST our idea, though I'd argue that the uniformly weak positive of the naive parameter model indicates that we are onto *something*, no matter how slight.

Perhaps the relationships between the tags can be better captured by other models. Let's try gradient boosting next, and then neural nets.

Huzzah! Onward!